# Open and Partitions the data to smaller chunks

The cell below will open the data and partition it into smaller chunks. As it is a large dataset, it is better to work with smaller chunks to avoid memory issues. To make the partitions, the program automatically detects the best way to split the data into smaller bits. It will save the partitions in the `partitions` folder. For each key, it will create a folder in the structure `partitions/key/`. Each folder will contain a parquet file for each unique Month-Year combination in the data.

Be aware that this process may take a while to run. The data from keys avgad and avgas are the same, jusf for diffent years.It may be a good idea to copy one to the other, but it is optional.

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import psutil
import os
import dask.dataframe as dd
import cupy
import geopandas as gpd
from dask import delayed
from dask.diagnostics import ProgressBar
from sklearnex import patch_sklearn
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import optuna
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
import os
import seaborn as sns
import scienceplots
plt.style.use('science', 'grid', 'notebook')
%matplotlib inline

In [ ]:

era_5_path = "your_path"
aquasat_path = "your_path"
keys = ["avgid", "avgas", "avgad", "avgd"]
output_path = "your_path"

def get_dataset_size(path):
    return os.path.getsize(path)

def get_available_memory():
    return psutil.virtual_memory().available

def calculate_optimal_partitions(dataset_size, available_memory, safety_factor=0.5):
    return max(1, int((dataset_size / available_memory) * safety_factor))

def open_datasets(era_5_path, aquasat_path, key):
    """
    Opens and filters datasets based on relevant dates.

    Parameters:
    era_5_path (str): The file path to the ERA-5 dataset.
    aquasat_path (str): The file path to the Aquasat dataset.
    key (str): The key to filter the ERA-5 dataset by.

    Returns:
    era_5_filtered (pandas.DataFrame): The filtered ERA-5 dataset.
    aquasat (dask.dataframe.DataFrame): The Aquasat dataset.
    relevant_dates (pandas.PeriodIndex): The unique relevant dates from the Aquasat dataset.
    """
    if os.path.exists(era_5_path) | os.path.exists(aquasat_path):
        raise FileNotFoundError("File not found")
    era_5_size = get_dataset_size(era_5_path)
    available_memory = get_available_memory()
    npartitions = calculate_optimal_partitions(era_5_size, available_memory)
    era_5 = xr.open_dataset(era_5_path, engine="cfgrib", chunks={}, filter_by_keys={'stepType': key})
    aquasat = dd.read_csv(aquasat_path)
    aquasat["time"] = dd.to_datetime(aquasat["time"]).dt.to_period("M")
    era_5 = era_5.to_dask_dataframe()
    era_5["time"] = era_5["time"].dt.to_period("M")
    relevant_dates = aquasat["time"].unique().compute()
    era_5_filtered = era_5.loc[era_5["time"].isin(relevant_dates)]
    era_5_filtered = era_5_filtered.repartition(npartitions=npartitions)
    return era_5_filtered, aquasat, relevant_dates

def process_and_save_partition(partition, output, key, relevant_dates):
    """
    Process and save a partition of data to Parquet files.

    Args:
        partition (pandas.DataFrame): The partition of data to process and save.
        output (str): The output directory where the Parquet files will be saved.
        key (str): The key used to identify the partition.
        relevant_dates (list): A list of relevant dates to filter the data.

    Returns:
        None
    """
    if not partition.empty:
        for date in relevant_dates:
            era_5_date = partition.loc[partition["time"] == date]
            if not era_5_date.empty:
                date_str = str(date).replace("/", "-")
                output_file = f"{output}{key}_era5/era_5_{date_str}.parquet"
                os.makedirs(os.path.dirname(output_file), exist_ok=True)
                era_5_date.to_parquet(output_file, index=True)

for key in keys:
    era_5, aquasat, relevant_dates = open_datasets(era_5_path, aquasat_path, key)
    # Apply the function to each partition
    era_5.map_partitions(process_and_save_partition, output=output_path, key=key,
                         relevant_dates=relevant_dates, meta=object).compute()

# Parquet treatment
This step will take the parquet files and pair 1:1 with the aquaset file. The ERA5 dataset contains way more entries thant the aquaset, so this may take a while. The output will be a csv, as it will be way smaller in size. The output will be saved in the format `key_era5_cvs/merged_{date}.csv`. 

In [ ]:

era_5_path = "your_path_to_era5_parquet_files"
aquasat_path = "your_path_to_aquasat.csv"
output_path = "your_output_path"
key = "your_key"
# Read the water data as a Dask DataFrame
aquasat = dd.read_csv(aquasat_path, assume_missing = True)
water["time"] = dd.to_datetime(water["time"]).dt.to_period("M")
relevant_dates = water["time"].unique().compute()

# Function to process each chunk
@delayed
def process_chunk(chunk, date_str, date, era_5_path, output_path, key = key):
    """
    Process a chunk of water data, perform a spatial join with ERA-5 data, and save the result to a CSV file.

    Parameters:
    chunk (pd.DataFrame): The chunk of water data to process.
    date_str (str): The date string used for file naming.
    date (pd.Period): The date to filter the data by.
    era_5_path (str): The path to the ERA-5 data files.
    output_path (str): The path to save the output CSV files.

    Returns:
    None
    """
    try:
        # Convert to GeoDataFrame
        df_water = gpd.GeoDataFrame(chunk, geometry=gpd.points_from_xy(chunk.longitude, chunk.latitude))
        df_water = df_water[df_water["time"] == date]

        # Read ERA-5 data
        era_5_file = f"{era_5_path}/era_5_{date_str}.parquet"
        if not os.path.exists(era_5_file):
            raise FileNotFoundError(f"ERA-5 file not found: {era_5_file}")
        
        df_era = dd.read_parquet(era_5_file).compute()
        df_era = gpd.GeoDataFrame(df_era, geometry=gpd.points_from_xy(df_era.longitude, df_era.latitude))

        # Perform spatial join
        df_merged = gpd.sjoin_nearest(df_water, df_era, how="left", distance_col="distance")

        # Save to CSV
        output_file = f"{output_path}key_era5_csv/merged_{date_str}.csv"
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        df_merged.to_csv(output_file, index=False)
    except Exception as e:
        print(f"Error processing chunk for date {date_str}: {e}")

# Process each date
tasks = []
for date in relevant_dates:
    date_str = str(date).replace("/", "-")
    # Process each partition of the Dask DataFrame
    for partition in water.to_delayed():
        task = process_chunk(partition, date_str, date)
        tasks.append(task)

# Execute the tasks with progress bar
with ProgressBar():
    dd.compute(*tasks)

# Preliminary Clean
This step makes a preliminary clean, of the date. It removes some uneeded collumns. The output will be saved in the format `key_era5_cvs/filtered_{date}.csv`.

In [ ]:

def read_water_data():
       """
       Reads water data from a CSV file and performs some preprocessing.

       Returns:
              water (dask.dataframe.DataFrame): A Dask DataFrame containing the water data.
       """
       water = dd.read_csv("aquasat_dat.csv", assume_missing=True)
       water["time"] = dd.to_datetime(water["time"]).dt.to_period("M")
       return water

def filter_merged_data(date, merged_path, merged_cleaned_path, output_name):
       """
       Filter the merged data based on certain conditions and save the filtered data to a CSV file.

       Parameters:
       - date (str): The date of the merged data file to be filtered.
       - merged_path (str): The path to the directory containing the merged data files.
       - merged_cleaned_path (str): The path to the directory where the filtered data will be saved.

       Returns:
       None
       """
       pre_filtered_data = dd.read_csv(f"{merged_path}/merged_{date}.csv")
       pre_filtered_data = pre_filtered_data[pre_filtered_data["doc"] >= 0]
       pre_filtered_data = pre_filtered_data[pre_filtered_data["type"] != "Facility"]
       post_filter = pre_filtered_data.drop(columns=["SiteID", "system:index", "TZID",
                                                        "source", "landsat_id",
                                                        "number", "step", "surface"])
       post_filter.to_csv(f"{merged_cleaned_path}{output_name}/filtered_{date}.csv", index=False,
                                      single_file=True)

def process_water_data():
       """
       Process water data by reading it, extracting relevant dates, and filtering merged data.

       Returns:
              None
       """
       water = read_water_data()
       relevant_dates = water["time"].unique().compute()
       merged_path = "merged_data"
       merged_cleaned_path = "merged_cleaned"
       output_name = "key_era5_csv"
       for date in relevant_dates:
              filter_merged_data(date, merged_path, merged_cleaned_path,
                                 output_name)
              

process_water_data()

# Merge all data
Merges all the csv files into a single file.

In [ ]:

def merge_data(input_path, output_path, output_name):
    """
    Merge the filtered data files into a single Parquet file.

    Parameters:
    - input_path (str): The path to the directory containing the filtered data files.
    - output_path (str): The path to the directory where the merged data will be saved.
    - output_name (str): The name of the output Parquet file.

    Returns:
    None
    """
    filtered_data = dd.read_csv(f"{input_path}/*.csv")
    filtered_data.to_csv(f"{output_path}/{output_name}.csv", index=False,
                         single_file=True)

# Removal of band values
In this steps we remove all the bands values that are over a absulte distance of a single standart deviation from the mean. This is done to remove outliers. In this step, we also subtract swir2 values from all the other band values and remove locations where the timediff is greater than the threshold.

In [ ]:

def compute_medians_and_vars(total_ds, bands):
    """
    Compute the medians and standard deviations for each band in the given dataset.

    Parameters:
    total_ds (Dataset): The dataset containing the bands.
    bands (list): A list of bands for which to compute the medians and standard deviations.

    Returns:
    dict: A dictionary where the keys are the bands and the values are tuples containing the median and standard deviation.

    """
    medians_vars = {}
    for band in bands:
        median = total_ds[band].mean().compute()
        std = total_ds[band].std().compute()
        medians_vars[band] = (median, std)
        print(f"Band: {band}, Median: {median}, std: {std}")
    return medians_vars

def create_filter_condition(total_ds, medians_vars):
    """
    Create a filter condition based on the median and standard deviation of each band in the dataset.

    Args:
        total_ds (pandas.DataFrame): The total dataset containing the bands.
        medians_vars (dict): A dictionary containing the median and standard deviation for each band.

    Returns:
        bool: The filter condition as a boolean value.

    """
    filter_condition = True
    for band, (median, std) in medians_vars.items():
        band_condition_1 = total_ds[band] <= (median + std)
        band_condition_2 = total_ds[band] >= (median - std)
        filter_condition = filter_condition & band_condition_1 & band_condition_2
    return filter_condition

def apply_filter(total_ds, filter_condition):
    """
    Apply a filter condition to a given dataset.

    Parameters:
    total_ds (pandas.DataFrame): The total dataset to filter.
    filter_condition (str): The condition to filter the dataset.

    Returns:
    pandas.DataFrame: The filtered dataset.
    """
    filtered_ds = total_ds[filter_condition]
    print(f"Filtered dataset length: {len(filtered_ds)}")
    return filtered_ds

def subtract_swir2(filtered_ds):
    """
    Subtract the 'swir1' band from each band in the given dataset.

    Args:
        filtered_ds (xarray.Dataset): The dataset containing the bands to be subtracted.

    Returns:
        xarray.Dataset: The dataset with each band subtracted by the 'swir1' band.
    """
    bands = ["blue", "green", "red", "swir1", "nir"]
    for band in bands:
        filtered_ds[band] = filtered_ds[band] - filtered_ds["swir2"]
    return filtered_ds

def filter_data(filtered_ds):
    """
    Filter the given dataset based on specific conditions.

    Parameters:
    filtered_ds (pandas.DataFrame): The dataset to be filtered.

    Returns:
    pandas.DataFrame: The filtered dataset.
    """
    filtered_ds = filtered_ds[filtered_ds["type"] != "Facility"]
    filtered_ds = filtered_ds[filtered_ds["doc"] >= 0]
    filtered_ds = filtered_ds[
        (filtered_ds['type'] != 'Lake') |
        ((filtered_ds['type'] == 'Lake') & (filtered_ds['timediff'].abs() <= 72))
    ]
    filtered_ds = filtered_ds[
        (filtered_ds['type'] != 'Stream') |
        ((filtered_ds['type'] == 'Stream') & (filtered_ds['timediff'].abs() <= 24))
    ]
    filtered_ds = filtered_ds[
        (filtered_ds['type'] != 'Estuary') |
        ((filtered_ds['type'] == 'Estuary') & (filtered_ds['timediff'].abs() <= 3))
    ]
    print(f"Final filtered dataset length: {len(filtered_ds)}")
    return filtered_ds

def process_data():
    """
    Process the data by performing various operations on the total_ds DataFrame.

    This function reads a CSV file called 'total_ds.csv' using Dask DataFrame,
    computes medians and variances for specified bands, creates a filter condition,
    applies the filter condition to the total_ds DataFrame, subtracts the 'swir1'
    column from the filtered DataFrame, and applies additional data filtering.

    The filtered DataFrame is then printed with the length and saved as a CSV file
    called 'filtered_ds.csv'.

    Returns:
        None
    """
    total_ds = dd.read_csv('total_ds.csv', assume_missing=True)
    bands = ["blue", "green", "red", "swir1", "swir2", "nir"]

    medians_vars = compute_medians_and_vars(total_ds, bands)
    filter_condition = create_filter_condition(total_ds, medians_vars)
    filtered_ds = apply_filter(total_ds, filter_condition)
    filtered_ds = subtract_swir2(filtered_ds)
    filtered_ds = filter_data(filtered_ds)
    filtered_ds = filtered_ds[filtered_ds["pixelCount"] >= 9]

    with ProgressBar():
        print(len(filtered_ds))
    filtered_ds.to_csv('filtered_ds.csv', index=False, single_file=True)

process_data()

# Removal of unecessary columns

In this step we remove all the columns that are not needed for the model. We also transform the DOC column into to a $\log_{10}$ scale. Any missing values are filled with the mean of the column.

In [ ]:

# Read the filtered dataset
total_ds_path = "your_path"
# filtered_ds.csv
filtered_ds = pd.read_csv("filtered_ds.csv")

# Drop unnecessary columns
columns_to_drop = [
    "Unnamed: 0", "date_unity", "path", "qa", "qa_sd", "row", "sat", ".geo",
    "endtime", "date", "chl_a", "p_sand", "secchi", "tis", "tss", "latitude_left",
    "longitude_left", "latitude_right", "longitude_right", "clouds", "time_right",
    "time_left", "geometry", "pwater", "id", "index_right", "valid_time",
    "blue_sd", "green_sd", "red_sd", "swir1_sd", "swir2_sd", "nir_sd", "pixelCount",
    "distance", "date_utc", "distance", "date_only", "type", "timediff"
]
filtered_ds = filtered_ds.drop(columns=columns_to_drop)

# Calculate wind speed
filtered_ds["wind"] = np.sqrt(filtered_ds["u10"]**2 + filtered_ds["v10"]**2)
filtered_ds = filtered_ds.drop(columns=["u10", "v10"])

# Process each column
for column in filtered_ds.columns:
    if column == "doc":
        continue
    print(f"Processing column: {column}")
    # Drop rows with missing values
    filtered_ds = filtered_ds.dropna(subset=[column])
    
    # Fill missing values with the column mean
    filtered_ds[column] = filtered_ds[column].fillna(filtered_ds[column].mean())
    
    # Replace remaining NaN values with the mean
    nan_mask = filtered_ds[column].isna()
    valid_data = filtered_ds[~nan_mask]
    mean_value = valid_data[column].mean()
    filtered_ds[column] = filtered_ds[column].fillna(mean_value)
    
# Apply logarithmic transformation to the "doc" column
filtered_ds["doc"] = np.log10(filtered_ds["doc"])

# Save the processed dataset
filtered_ds.to_csv("machine_learning_ds.csv", index=False)


# SVR MODEL

In this step we train the SVR model. We use optuna to find the best hyperparameters for the model. The output will be the best hyperparameters and the model itself. It will be created a database to track the model evolution, and can be done by running the command `optuna-dashboard sqlite:///db.sqlite3` in the dash board. The best model will pe pickled and saved in the `models` folder. It is using the sklearnx to train, which is a version of sklearn made by intel to optimize the training.

In [ ]:
data_path = "your_path"
# machine_learning_ds.csv
patch_sklearn()

storage = "sqlite:///db.sqlite3"
X_scaler = StandardScaler(data_path)
data = pd.read_csv('')
X = data.drop('doc', axis=1)
y = data['doc']


# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=42)
kf_inner = KFold(n_splits=10, shuffle=True, random_state=42)
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def objective(trial):
    # Parameters
    params = {
        'C': trial.suggest_float('C', 0.1, 10),
        'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),
        'tol': trial.suggest_float('tol', 1e-5, 1e-3),
        'epsilon': trial.suggest_float('epsilon', 0.1, 1.0),
        'shrinking': trial.suggest_categorical('shrinking', [True, False]),
    }
    # Model
    model = SVR(**params, kernel='rbf', cache_size=1024)
    # Cross Val
    score = cross_val_score(model, X_train, y_train, cv=kf_inner.split(X_train),
                                n_jobs=-1, scoring='r2')
    score = score.mean()
    return score


# Study
study = optuna.create_study(direction='maximize', study_name='svr_study',
                            storage=storage, load_if_exists=True,
                            sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials = 100 ,n_jobs=-1, show_progress_bar=True)
# Best Params
print(f'Best Params: {study.best_params}')
print(f'Best Score: {study.best_value}')

# Model
model = SVR(**study.best_params)
model.fit(X_train, y_train)


# pred
y_pred = model.predict(X_test)

# Evaluate
#rmse = root_mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
cross_val = cross_val_score(model, X, y, cv=10, n_jobs=-1, scoring='r2',
                            verbose=2)
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'R2 Score: {r2}')
print(f'Cross Val Score: {cross_val}')
print(f'Cross Val Mean: {cross_val.mean()}')
print(f'Cross Val Std: {cross_val.std()}')

# Save the model
os.mkdir('models')
joblib.dump(model, 'models/svr_model.pkl')


# Random Forest Model

In this step we train the Random Forest model. We use optuna to find the best hyperparameters for the model. The output will be the best hyperparameters and the model itself. It will be created a database to track the model evolution, and can be done by running the command `optuna-dashboard sqlite:///db.sqlite3` in the dash board. The best model will pe pickled and saved in the `models` folder. It is using the sklearnx to train, which is a version of sklearn made by intel to optimize the training.

In [ ]:
patch_sklearn()
data_path = 'your_path'
# machine_learning_ds.csv
storage = "sqlite:///db.sqlite3"

data = pd.read_csv(your_path)
X = data.drop('doc', axis=1)
y = data['doc']
X_scaler = StandardScaler()


# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=42)
kf_inner = KFold(n_splits=10, shuffle=True, random_state=42)
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)


def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def objective(trial):
    # Parameters
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 4000),
        'max_depth': trial.suggest_int('max_depth', 3, 600),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 200),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 200),
        'max_features': trial.suggest_categorical('max_features',
                                                 ['sqrt', 'log2']),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 1000),
    }
    
    
    # Model
    model = RandomForestRegressor(**params)
    # Score 
    score = cross_val_score(model, X_train, y_train, cv=kf_inner,
                            scoring='r2', n_jobs=-1)
    score = score.mean()
    return score



# Study
study = optuna.create_study(study_name='rf_study', storage=storage,
                            direction='maximize', load_if_exists=True,
                            sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=100, n_jobs=-1, show_progress_bar=True)
# Best Params
print(f'Best Params: {study.best_params}')
print(f'Best Score: {study.best_value}')

# Model
model = RandomForestRegressor(**study.best_params)
model.fit(X_train, y_train)


# pred
y_pred = model.predict(X_test)

# Evaluate
#rmse = root_mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
cross_val = cross_val_score(model, X, y, cv=10, n_jobs=-1, scoring='r2',
                            verbose=2)
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'R2 Score: {r2}')
print(f'Cross Val Score: {cross_val}')
print(f'Cross Val Mean: {cross_val.mean()}')
print(f'Cross Val Std: {cross_val.std()}')

# Save the model
os.mkdir('models')
joblib.dump(model, 'rf_model.pkl')


# XGBoost Model

In this step we train the XGBoost model. We use optuna to find the best hyperparameters for the model. The output will be the best hyperparameters and the model itself. It will be created a database to track the model evolution, and can be done by running the command `optuna-dashboard sqlite:///db.sqlite3` in the dash board. The best model will pe pickled and saved in the `models` folder. This model is trained using the GPU.

In [ ]:
data_path = "your_path"
# machine_learning_ds.csv
storage = "sqlite:///db.sqlite3"
data = pd.read_csv(data_path)
rs = 42
folds = 10
n_iter = 10
X_scaler = StandardScaler()

X = data.drop('doc', axis=1)
y = data["doc"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,
                                                    test_size=0.3)

kf = KFold(n_splits=10, shuffle=True, random_state=42)
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)


def objective(trial):
    params = {
        'normalize_type': trial.suggest_categorical('normalize_type', ['tree', 'forest']),
        'rate_drop': trial.suggest_float('rate_drop', 0.1, 0.9),
        'skip_drop': trial.suggest_float('skip_drop', 0.1, 0.9),
        'eta': trial.suggest_float('eta', 0.01, 0.9),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 400),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'subsample': trial.suggest_float('subsample', 0.5, 1),
    }

    xgbr = XGBRegressor(tree_method='hist', eval_metric='rmse',
                        objective='reg:squarederror', device="cuda",
                        verbosity=1, booster='dart', **params)

    rmse = cross_val_score(xgbr, X_train, y_train, cv=kf.split(X_train, y_train),
                           scoring='r2')

    return rmse.mean()

study = optuna.create_study(direction='maximize', study_name='XGBoost',
                            storage=storage, load_if_exists=True,
                            sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=100, show_progress_bar=True)

# Train model with best hyperparameters
best_params = study.best_params
model = XGBRegressor(tree_method='hist', eval_metric='rmse',
                           objective='reg:squarederror', device="cuda",
                            verbosity=1, booster='dart', **best_params)
model.fit(X_train, y_train)



# Best hyperparameters
print(study.best_params)
print(study.best_value)
print(study.best_trial)


# Evaluate model
y_pred = model.predict(cupy.array(X_test))

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
cross_val = cross_val_score(model, cupy.array(X_test), y_test, cv=10, scoring='r2',
                            verbose=2)
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'R2 Score: {r2}')
print(f'Cross Val Score: {cross_val}')
print(f'Cross Val Mean: {cross_val.mean()}')
print(f'Cross Val Std: {cross_val.std()}')


model.save_model('XGBoost.json')

# Get Scatter plot from models

We will get the scatter plot from the models.

In [ ]:
df_path = 'your_path'
#machine_learning_ds.csv
models = ['svr_model.pkl', 'rf_model.pkl', 'XGBoost.model']
titles = ['SVR', 'Random Forest', 'XGBoost']
# Random state
rs = 42
# Load data
df = pd.read_csv(df_path)
X = df.drop("doc", axis=1)
y = df["doc"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=rs)

def plot_seaborn(y_test, y_pred, title, ax=None):
    y_pred = 10 ** y_pred
    y_test = 10 ** y_test
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    bias = np.mean( - y_test + y_pred)
    print(f"RMSE of {title}: {rmse}")
    print(f"MAE of {title}: {mae}")
    print(f"Bias of {title}: {bias}")
    y_pred = np.log10(y_pred)
    y_test = np.log10(y_test)
    r2 = r2_score(y_test, y_pred)
    print(f"R2 Score of {title}: {r2_score(y_test, y_pred)}")
    g = sns.regplot(x=y_test, y=y_pred, scatter_kws={'alpha':0.7, 'color':'b'},
                    line_kws={'color':'r', 'lw':2}, ax=ax)
    g.set(xlabel='Real', ylabel='Predição')
    g.set_title(title)
    g.set_xticks(np.arange(-1, 2, 1), [fr"$10^{{{loc}}}$" for loc in np.arange(-1, 2, 1)])
    g.set_yticks(np.arange(-1, 2, 1), [fr"$10^{{{loc}}}$" for loc in np.arange(-1, 2, 1)])
    g.plot([y_test.min(), 2], [y_test.min(), 2], 'k--', lw=2)
    g.set_xlim(-1, 2)
    g.set_ylim(-1, 2)
    g.annotate(fr"$R^{2}: {r2:.2f}$", xy=(0.15, 0.9), xycoords='axes fraction',
                fontsize=16, ha='center', va='center')
    return g
fig, axs = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)
for model, title, ax in zip(models, titles, axs):
    if model == 'XGBoost.model':
        model = XGBRegressor()
        model.load_model('XGBoost.json')
    else:
        model = joblib.load(model)
    y_pred = model.predict(X_test)
    plot_seaborn(y_test, y_pred, title, ax=ax)
plt.show()

# Feature Importance

We will get the feature importance from the models. We use the permutation importance to get the feature importance.

In [ ]:

csv_path = "your_trainning_ds.csv"
plt.style.use(['science', 'notebook', 'grid'])
data = pd.read_csv(csv_path)
models = ['svr_model.pkl', 'rf_model.pkl', 'XGBoost.model']
titles = ['SVR', 'Random Forest', 'XGBoost']
X = data.drop('doc', axis=1)
y = data['doc']
X_scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=42)
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)
model = joblib.load('rf_model.pkl')

def plot_feature_importance(model, X, y, title, ax=None, scoring=None):
    result = permutation_importance(model, X, y, n_repeats=10,
                                    random_state=42, n_jobs=-1,
                                    scoring=scoring)
    results = pd.DataFrame(result.importances_mean, index=X.columns,
                            columns=['importance'])
    #results.sort_values(by='importance', inplace=True)
    results.plot.bar(yerr=result.importances_std, ax=ax, legend=False)
    ax.set_xlabel('Importance')
    ax.set_ylabel('Features')
    ax.set_title('{} Feature Importance'.format(title))
    return results
fig, axs = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)
for model, title, ax in zip(models, titles, axs):
    if model == 'XGBoost.model':
        model = XGBRegressor()
        model.load_model('XGBoost.json')
    else:
        model = joblib.load(model)
    if title == 'SVR':
        scoring = 'neg_mean_squared_error'
    else:
        scoring = None
    plot_feature_importance(model, X_test, y_test, title, ax=ax, scoring=scoring)



# Get some visualizations

Get some visualizations from the data, to compare with the original article

In [ ]:

csv_path = "your_path_to_ds.csv"

data = pd.read_csv(csv_path)

bands_dict = {
    "blue": "b", "green": "g", "red": "r", "nir": "m", "swir1": "y", "swir2": "k"
}
    

def plot_band_histograms(data, bands_dict):
    """
    Plots histograms for each band in the given DataFrame.

    Parameters:
    - data (pandas.DataFrame): The DataFrame containing the bands data.
    - bands_dict (dict): A dictionary mapping band names to colors.

    Returns:
    None
    """
    fig, axs = plt.subplots(2, 3, figsize=(15, 10), constrained_layout=True)
    for band, ax, color in zip(bands_dict.keys(), axs.flatten(), bands_dict.values()):
        ax.hist(data[band], bins=50, color=color, alpha=0.7)
        ax.set_title(f"{band.capitalize()} Band")
        ax.set_xlabel("Pixel Value")
        ax.set_ylabel("Frequency")
        ax.axvline(data[band].mean(), color='k', linestyle='dashed', linewidth=1)
        ax.axvline(data[band].mean() + data[band].std(), color='k',
                   linestyle='dashed', linewidth=1)
        ax.axvline(data[band].mean() - data[band].std(), color='k',
                     linestyle='dashed', linewidth=1)
        ax.annotate(fr"$\mu: {data[band].mean():.2f}$" + "\n" +
                    fr"$\sigma: {data[band].std():.2f}$",
                    xy=(0.75, 0.9), xycoords='axes fraction',
                    ha='center', va='center', fontsize=16)
        
    plt.show()
def plot_doc_distribution(data, color = 'b'):
    """
    Plots the distribution of the 'doc' column in the given DataFrame.

    Parameters:
    - data (pandas.DataFrame): The DataFrame containing the 'doc' column.
    - color (str): The color to use for the plot.

    Returns:
    None
    """
    plt.figure(figsize=(8, 6))
    plt.hist(data['doc'], bins=50, color=color, alpha=0.7,
             edgecolor='k', linewidth=1.5)
    plt.title("Distribution of DOC Values")
    plt.xlabel("DOC Value")
    plt.ylabel("Frequency")
    plt.annotate(fr"$\mu: {data['doc'].mean():.2f}$" + "\n" +
                 fr"$\sigma: {data['doc'].std():.2f}$" + "\n" +
                 fr"$\eta: {data['doc'].median():.2f}$",
                 xy=(0.75, 0.9), xycoords='axes fraction',
                 ha='center', va='center', fontsize=16)
    plt.xticks(np.arange(-1, 3, 1), [fr"$10^{{{loc}}}$" for loc in np.arange(-1, 3, 1)])
    plt.show()

plot_band_histograms(data, bands_dict)
plot_doc_distribution(data)

# Plot Summary statistics

Plot all the summary statistics from the data.

In [ ]:

sns.color_palette("hls", 8)
data_path = "your_path_to_ds.csv"
data = pd.read_csv(data_path)
def plt_distribution(x, ax, title, var_name, kde = False, color = 'b'):
    sns.histplot(x, kde=kde, ax=ax, color=color)
    mean = x.mean()
    std = x.std()
    median = x.median()
    ax.set_title(title)
    ax.set_xlabel(var_name)
    ax.set_ylabel('Frequency')
    ax.annotate(fr"$\mu: {mean:.2E}$", xy=(0.8, 0.95), xycoords='axes fraction',
                 fontsize=16, ha = "center", va="center")
    ax.annotate(fr"$\sigma: {std:.2E}$", xy=(0.8, 0.9), xycoords='axes fraction',
                    fontsize=16, ha = "center", va="center")
    ax.annotate(fr"$\eta: {median:.2E}$", xy=(0.8, 0.85), xycoords='axes fraction',
                fontsize=16, ha = "center", va="center")
    return ax

X = data.drop('doc', axis=1)
num_var = X.columns
fig, axs = plt.subplots(4, 4, figsize=(30, 30), constrained_layout=True,
                        dpi=300)
for var, ax in zip(num_var, axs.ravel()):
    plt_distribution(data[var], ax, var, var_name=var)
    mean = data[var].mean()
    std = data[var].std()
    median = data[var].median()
if len(num_var) != len(axs.ravel()):
    for ax in axs.ravel()[len(num_var):]:
        ax.set_visible(False)

In [ ]:

data_path = "your_path_to_ds.csv"
data = pd.read_csv(data_path)
# this data need to have this collumns. If you didnt
# change the names, you can import the "filtered_ds.csv"
    
data["date_utc"] = pd.to_datetime(data["date_utc"])
date = data["date_utc"]
year = date.dt.year
month = date.dt.month
hour = date.dt.hour

landsat = data["sat"]
location = data["type"]
fig, ax = plt.subplots(5, 1, figsize=(16, 16), constrained_layout=True)
ax[0].hist(year, bins=35, color='b', alpha=0.7,
            histtype='bar', ec='black')
ax[0].set_title("Year")
ax[0].set_xlabel("Year")
ax[0].set_ylabel("Frequency")
ax[1].hist(month, bins=12, color='r', alpha=0.7,
            histtype='bar', ec='black')
ax[1].set_title("Month")
ax[1].set_xlabel("Month")
ax[1].set_ylabel("Frequency")

ax[2].hist(hour, bins=24, color='g', alpha=0.7,
            histtype='bar', ec='black')
ax[2].set_title("Hour")
ax[2].set_xlabel("Hour")
ax[2].set_ylabel("Frequency")

ax[3].bar(landsat.value_counts().index, landsat.value_counts().values, color='purple', alpha=0.7)
ax[3].set_title("Landsat")
ax[3].set_xlabel("Landsat Satellite")
ax[3].set_ylabel("Frequency")

ax[4].bar(location.value_counts().index, location.value_counts().values, color='orange', alpha=0.7)
ax[4].set_title("Location of Sample")
ax[4].set_xlabel("Location")
ax[4].set_ylabel("Frequency")

ax[0].set_xticks(np.arange(1984, 2020, 5))
ax[0].set_xticklabels([str(i) for i in np.arange(1984, 2020, 5)])

ax[1].set_xticks(np.arange(1, 13, 1))
months_name = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
                'Oct', 'Nov', 'Dec']
ax[1].set_xticklabels(months_name)

ax[2].set_xticks(np.arange(0, 24, 1))


ax[3].set_xticks([5, 7, 8])
landsat_name = ['Landsat 5', 'Landsat 7', 'Landsat 8']

ax[3].set_xticklabels(landsat_name)
ax[3].set_xlabel("Landsat Satellite")

ax[4].set_xticks([0, 1, 2])
location_name = ['Stream', 'Lake', 'Estuary']
ax[4].set_xticklabels(location_name)
ax[4].set_xlabel("Location of Sample")



# Plot trainning curve from models

Plot the trainning curve from the models.

In [ ]:

so.Plot.config.theme.update(sns.axes_style('whitegrid') | sns.plotting_context('notebook'))
plt.style.use(['science', 'notebook', 'grid'])
titles = ['SVR', 'Random Forest', 'XGBoost']
storage = "sqlite:///db.sqlite3"
studies = ['svr_study', 'rf_study', 'XGBoost']

f = mpl.figure.Figure(figsize=(16, 9))
sf1, sf2, sf3 = f.subfigures(1, 3)

svr_study = optuna.load_study(study_name="svr_study", storage=storage)
df = svr_study.trials_dataframe()
(so.Plot(data=df, x='number', y='value')
    .add(so.Dot(marker='o'))
    .add(so.Line(color='blue'), so.PolyFit(order=2))
    .label(title="SVR optimization", x_label='Number of Trial',
           y_label='Objective Value')
    .on(sf1)
    .plot())

rf_study = optuna.load_study(study_name="rf_study", storage=storage)
df = rf_study.trials_dataframe()
(so.Plot(data=df, x='number', y='value')
    .add(so.Dot(marker='o'))
    .add(so.Line(color='blue'), so.PolyFit(order=3))
    .label(title="Random Forest optimization", x_label='Number of Trial',
           y_label='Objective Value')
    .on(sf2)
    .plot())

XGB_study = optuna.load_study(study_name="XGBoost", storage=storage)
df = XGB_study.trials_dataframe()
(so.Plot(data=df, x='number', y='value')
    .add(so.Dot(marker='o'))
    .add(so.Line(color='blue'), so.PolyFit(order=3))
    .label(title="XGBoost optimization", x_label='Number of Trial',
           y_label='Objective Value')
    .on(sf3)
    .plot())
